以一个电商的例子记录如何从0到1建立用户画像。

企业在运营过程中积累了大量经营数据和用户行为数据，建立企业的用户画像简单来说就是给企业每个用户打上各种“标签”，利用这个标签系统就能有效地进行精细化运营。

这里利用Python，SQL等技术手段整合业务数据、埋点数据和日志数据等，并建立用户画像的数据仓库。利用数据仓库进行经营分析，营销优化，以及个性化推荐等服务。

这里通过一个实例讲解用户画像的４个部分。本文是用户画像的第一部分“项目目标与数据收集”。

1. 项目目标与数据收集
2. 数据指标体系设计
3. 用户画像数据仓库开发
4. 用户画像的应用

# 项目目标与数据收集

## 项目目标
某中小型电商网站有几十万的用户，销售各类化妆品。用户可以在电商网站浏览，收藏或者下单等。现在随着用户的增长，电商平台数据仓库积累了大量**业务数据、埋点数据和日志数据**。公司希望挖掘数据价值，建立用户画像提高运营效率，理解用户需求，为用户提供更好的购物体验。
所以我们的目标是**1.建立用户画像数据仓库。2.如何利用用户画像给公司带来价值。**对于第二点，我们会在实例的第四部分展开。

首先我们先了解下，用户标签数据仓库的实现的框架。

![frame](pic/user_profile_frame.png)

公司在运营中使用后台数据库记录运营数据，通过网页数据埋点和日志收集用户行为数据。

通过ETL(Extract-Transform-Load)获取，整合并处理来自不同数据源的数据后存入公司数据仓库。数据仓库中的数据可用于分析，建模，可视化等操作为公司创造价值。数据仓库可以理解为存储所有部门可以的通用型分析数据库。

接下来，再次通过ETL对数据仓库中的数据进行处理，并存入用户画像数据仓库。运营人员和数据分析师就能使用标签化的用户数据进行高效运营和分析。比如对高价值用户群体提供VIP服务，更具用户消费喜好精准推送广告促销，针对用户资料的不同话术等。

## 数据收集与处理

本例的数据来自Kaggle的[eCommerce Events History in Cosmetics Shop数据集](https://www.kaggle.com/mkechinov/ecommerce-events-history-in-cosmetics-shop)。数据集包含了一家销售化妆品的网络电商从2019年十月到2020年二月的用户行为数据。数据结构如下图所示。

![events_table_all](pic/events_all_tb.png)


为了进行用户标签建模，我们第一步是使用以上电商数据集创建一个模拟数据仓库。这里我用本地数据库[PostgreSQL](https://www.postgresql.org/)模拟公司数据仓库和用户画像数据仓库。但在实际工作中需要根据需求使用适合工具，比如对于有巨大数据量选择分布式数据仓库hive。

数据仓库的建立可以分为以下几步：

第一步，在PostgreSQL中[创建一个test用户](https://www.postgresql.org/docs/8.0/sql-createuser.html)，然后[创建一个user_profile_dw的数据库](https://www.postgresql.org/docs/9.0/sql-createdatabase.html)并把test用户设置为owner。

第二步，使用Python连接数据库并根据以下数据模型创建表格。这个简化的数据仓库中包含2个事实表(Fact table): orders, events，和3个维度表(Dimension table): users, products, time。此外，需要额外创建一个临时表格staging_events用于存放导入的csv数据。

![erd](pic/dw_erd.png)

第三步，我们现在只有一个用户行为数据，所以我们需要用这个数据去创建其他5个表格。使用Python把csv数据导入临时表staging_events。然后利用SQL和Python对数据进行处理并插入对应表格。**考虑到本例目的为分享用户画像构建，所以只随机抽取1000个产品作相关的数据以提高代码运行速度。**

这里简单展开下如何使用Python和SQL插入数据到events, products, orders, 和users表格。

**事实表events的数据导入**。

events记录所有用户的行为，比如浏览，加入购物车，下单等。在实际工作中，events表可以通过公司后台数据库、埋点数据和日志数据生成。

首先从staging_events中随机选取1000个产品，然后根据这1000个产品选取对应events和所需的列。
```postgresql
--　设置seed使得每次随机生成的数据一致。
SELECT setseed(0);
--　从临时表staging_events中选取需要的列并插入events中。
INSERT INTO events (
                    event_time,
                    event_date,
                    event_type,
                    product_id,
                    price,
                    user_id,
                    user_session)
SELECT
    event_time,
    DATE(event_time) AS event_date,
    event_type,
    product_id,
    price,
    user_id,
    user_session
FROM staging_events S
-- 随机选取1000个商品
WHERE EXISTS (
    SELECT 1
    FROM (
        SELECT DISTINCT
            PRODUCT_ID
        FROM staging_events
        WHERE random()<0.01
        LIMIT 1000
    ) T1
    WHERE T1.PRODUCT_ID = S.PRODUCT_ID);
```

**维度表products的数据导入**。

维度表products包含产品相关信息，比如产品名称，类别，品牌名等。在实际工作中，可由公司后台数据通过ETL存入数据仓库。

本例生成products先从staging_events中随机选取1000个产品，再根据这1000个产品选取对应的产品列。由于在staging_events中同个产品有category_code或brand缺失的情况，所以使用窗口函数以brand, category_code排序，如果同个产品在不同行有brand或category_code，有数据的行会排在前面，然后我们只需选取第一个。
```postgresql
--　设置seed使得每次随机生成的数据一致。
SELECT setseed(0);
INSERT INTO products
SELECT
    product_id,
    category_id,
    category_code,
    brand
-- 
FROM (
    SELECT
        s.product_id,
        category_id,
        category_code,
        brand,
        -- 以产品id为组，按照brand, categroy_code排序，有数据的行会被排在前面。
        ROW_NUMBER() OVER(PARTITION BY product_id
                          ORDER BY brand, category_code) as row_num
    FROM staging_events S
    -- 随机选取1000个商品
    WHERE EXISTS (
        SELECT 1
        FROM (
            SELECT DISTINCT
                PRODUCT_ID
            FROM staging_events
            WHERE random()<0.01
            LIMIT 1000
        ) T1
        WHERE T1.PRODUCT_ID = S.PRODUCT_ID
)) T2
WHERE ROW_NUM = 1;
```

**事实表orders的数据导入**。

事实表orders包含所有用户的购买信息，在实际工作中从公司后台数据库通过ETL存入数据仓库。

由于本例数据只来自用户行为数据，我们使用events中```event_type = 'purchase'```获取用户购买信息，并且使用user_session作为订单号(so_)本例我们从staging_events中随机选取1000个产品，再根据这1000个产品选取对应的产品列。由于在staging_events中同个产品有category_code或brand缺失的情况，所以使用窗口函数以brand, category_code排序，如果同个产品在不同行有brand或category_code，有数据的行会排在前面，然后我们只需选取第一个。
```postgresql
INSERT INTO orders(po_number,
                   product_id,
                   user_id,
                   so_created_time,
                   so_created_date,
                   price,
                   qty)
SELECT
    po_number,
    product_id,
    user_id,
    so_created_time,
    so_created_date,
    price,
    qty
FROM (
        SELECT
            user_session as po_number,
            product_id,
            user_id,
            price,
            count(*) as qty
        FROM events
        WHERE event_type = 'purchase'
        GROUP BY po_number, product_id, user_id, price
) tbo
INNER JOIN (
        SELECT
        *
        FROM
        (
            SELECT
                user_session,
                event_time as so_created_time,
                event_date as so_created_date,
                ROW_NUMBER() OVER(PARTITION BY user_session
                                  ORDER BY event_time desc) AS ROW_NUM
            FROM events
            WHERE event_type = 'purchase'
        ) T1
        WHERE ROW_NUM = 1
) tbt ON tbo.po_number = tbt.user_session;
```

In [ ]:
# 